## 数组切片、过滤、相关性与协同过滤

In [1]:
import numpy as np

### 切片

In [4]:
a=np.array([[2,1,4,6,2],
            [3,2,3,4,5],
            [2,1,4,5,6]])
a[2,:]


array([2, 1, 4, 5, 6])

In [3]:
a[:,[1,3]]

array([[1, 6],
       [2, 4],
       [1, 5]])

### 过滤
* where\delete

In [6]:
a

array([[2, 1, 4, 6, 2],
       [3, 2, 3, 4, 5],
       [2, 1, 4, 5, 6]])

In [7]:
a>3 #结果为布尔变量

array([[False, False,  True,  True, False],
       [False, False, False,  True,  True],
       [False, False,  True,  True,  True]])

In [8]:
#选择所有满足条件的元素，但是原来的数组形状变为一维向量
a[a>2]

array([4, 6, 3, 3, 4, 5, 4, 5, 6])

In [10]:
#选择满足条件元素的位置，0号位为行标，1号位为列标
np.where(a>2)

(array([0, 0, 1, 1, 1, 1, 2, 2, 2], dtype=int64),
 array([2, 3, 0, 2, 3, 4, 2, 3, 4], dtype=int64))

In [11]:
#确定所有满足条件的位置，并把不满足条件的元素用0替代
np.where(a>3,a,0)

array([[0, 0, 4, 6, 0],
       [0, 0, 0, 4, 5],
       [0, 0, 4, 5, 6]])

In [12]:
#删除对应行，注意axis=0决定了列表中[0,2]为行标
np.delete(a, [0], axis=0) #删除第0行

array([[3, 2, 3, 4, 5],
       [2, 1, 4, 5, 6]])

In [14]:
#axis=1，则此时为列标 
np.delete(a, [0], axis=1)#删除第0列

array([[1, 4, 6, 2],
       [2, 3, 4, 5],
       [1, 4, 5, 6]])

In [15]:
#axis=1，则此时为列标
np.delete(a, [0, 2], axis=1) #删除第0,2列

array([[1, 6, 2],
       [2, 4, 5],
       [1, 5, 6]])

In [19]:
np.where(a>5)[1] # 取第1个点的行列坐标

array([3, 4], dtype=int64)

In [20]:
np.delete(a, np.where(a>5)[1], axis=1) 

array([[2, 1, 4],
       [3, 2, 3],
       [2, 1, 4]])

### 相关性
* np.cov(x)计算协方差矩阵
* np.corrcoef(x)相关系数矩阵——皮尔森(Pearson)系数
* 如果x为一个二维数组，则计算出来的矩阵为每一行之间的相关系数

In [21]:
a=[2,1,4]
b=[3,2,4]
x=np.array([a,b])
x

array([[2, 1, 4],
       [3, 2, 4]])

In [23]:
np.cov(a,b)

array([[2.33333333, 1.5       ],
       [1.5       , 1.        ]])

In [26]:
np.cov(x)

array([[2.33333333, 1.5       ],
       [1.5       , 1.        ]])

In [24]:
np.corrcoef(a,b)

array([[1.        , 0.98198051],
       [0.98198051, 1.        ]])

In [25]:
np.corrcoef(x)

array([[1.        , 0.98198051],
       [0.98198051, 1.        ]])

In [27]:
np.corrcoef(x)[0,1]

0.9819805060619656

### 协同过滤推荐算法的numpy实现
基于物品的协同过滤算法（基于用户的其实也非常类似）:旨在寻找相似的物品，然后向目标用户推荐其已购买的物品的相似物品。

similarity: 提取所有用户对Item1, Item2, Item3，Item4四个物品的评分，每个物品对应一条评分向量，向量间的相似度就是物品间的相似度。(注意计算向量间相似度时，必须元素对应，即某个用户必须同时对两个物品进行了评分)

prediction: 假设目标用户购买了Item1, Item2，未购买Item3和Item4。那么Item3对目标用户的吸引力计算：是以Item1和Item2与Item3的相似度为权重，用户对Item1和Item2的评分加权均值即为目标用户对Item3的吸引力。

相似度：pearson相关系数（最常用的）


In [29]:
#每一行为某个用户对所有item的评分（比如豆瓣用户对电影的评分，0值意味着此用户没有对此item做过评分）
data = np.array([[4.0, 2.0, 0.0, 2.0, 0.0, 2.0],
                [3.5, 2.5, 3.0, 3.5, 2.5, 3.0],
                [3.0, 0.0, 2.5, 5.0, 3.0, 0.0],
                [2.5, 2.5, 0.0, 3.5, 4.0, 0.0],
                [2.5, 2.0, 3.5, 0.0, 3.5, 2.0],
                [0.0, 3.0, 4.0, 3.0, 3.0, 2.0],
                [4.5, 1.5, 3.0, 5.0, 3.5, 0.0]])

In [30]:
#计算1,2两个物品的相似度（这里的编号和索引一致从0开始的）
items = data[:, [1, 2]]
print(items)


[[2.  0. ]
 [2.5 3. ]
 [0.  2.5]
 [2.5 0. ]
 [2.  3.5]
 [3.  4. ]
 [1.5 3. ]]


In [34]:
#找到有0元素的位置
nn=np.where(items == 0)
nn

(array([0, 2, 3], dtype=int64), array([1, 0, 1], dtype=int64))

In [36]:
items1 = np.delete(items, nn[0],axis=0)
items1

array([[2.5, 3. ],
       [2. , 3.5],
       [3. , 4. ],
       [1.5, 3. ]])

In [40]:
np.corrcoef(items1.T)[0,1]

0.674199862463242

In [39]:
# 计算物品之间的相似矩阵

def similarity(a,b,data):
    items = data[:, [a, b]]
    mm = np.where(items == 0)
    items1 = np.delete(items, nn[0],axis=0)
    if items1.size == 0:
        return 0
    else:
        return np.corrcoef(items1.T)[0,1]

In [41]:
print(similarity(1,2,data))

0.674199862463242


In [44]:
#存储所有相关系数
n1 = data.shape[1]#items的数量
simiMat = np.ones((n1,n1))
simiMat

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [45]:
# 对所有items进行循环
for i in range(0,n1):
    for j in range(i+1,n1):
        r1 = similarity(i,j,data)
        simiMat[i,j] = r1
        simiMat[j,i] = r1
simiMat        

array([[ 1.        , -0.83565784, -0.96904869,  0.37562554,  0.15775211,
        -0.39438875],
       [-0.83565784,  1.        ,  0.67419986, -0.15393727, -0.67419986,
         0.71818485],
       [-0.96904869,  0.67419986,  1.        , -0.43589484,  0.09090909,
         0.20751434],
       [ 0.37562554, -0.15393727, -0.43589484,  1.        , -0.22832587,
        -0.45801528],
       [ 0.15775211, -0.67419986,  0.09090909, -0.22832587,  1.        ,
        -0.76088591],
       [-0.39438875,  0.71818485,  0.20751434, -0.45801528, -0.76088591,
         1.        ]])

###  计算items 3 对 用户 4 的吸引力

In [48]:

user=data[4,:]
user

array([2.5, 2. , 3.5, 0. , 3.5, 2. ])

In [49]:
nn=np.where(user>0)
nn

(array([0, 1, 2, 4, 5], dtype=int64),)

In [51]:
s1=simiMat[3,:][nn[0]]
s1

array([ 0.37562554, -0.15393727, -0.43589484, -0.22832587, -0.45801528])

In [52]:
#norm求数组的L1范数，也就是绝对值的和
sum(user[nn[0]]*s1)/np.linalg.norm(s1, 1)

-1.5798617306293956

### 计算用户4所有未评分的商品对其的吸引力

In [54]:
def xiyin(k,data,s1):
    user = data[k,:]
    n1=np.where(user>0)
    n2=np.where(user==0.)#只需要未评分的商品，所以做此过滤
    l1=[]
    for i in n2[0]:
        s2=s1[i,:][n1[0]]
        l1.append(sum(user[n1[0]]*s2)/np.linalg.norm(s2, 1))
    return n2[0],l1
s1=simiMat
print(xiyin(0,data,s1))

(array([2, 4], dtype=int64), [-1.3052045285761362, -1.4802704202310857])
